In [ ]:
%pip install -q -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *

Connection to MySQL

In [ ]:
mysql_user = "flosrv"
password = "Nesrine123"
host = "localhost"
port = 3306
database = "Oceanography_data_analysis"
metadata = MetaData()
# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{password}@{host}/{database}", isolation_level ='AUTOCOMMIT')

Get Available Stations ID List

In [4]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

146


In [5]:
def afficher_info_bouees_aleatoires(buoy_datas, prefix=None, df_wanted=None):
    """
    Affiche les informations de DataFrames (marine et/ou météo) de deux bouées choisies aléatoirement.
    """
    # Sélection de deux bouées au hasard
    sample_buoy_id_marine = random.choice(list(buoy_datas.keys()))
    sample_buoy_id_meteo = random.choice(list(buoy_datas.keys()))

    # Choix du préfixe (ex: "Cleaned")
    marine_key = f"{prefix} Marine" if prefix else "Marine"
    meteo_key = f"{prefix} Meteo" if prefix else "Meteo"

    sample_marine_df = buoy_datas[sample_buoy_id_marine].get(marine_key)
    sample_meteo_df = buoy_datas[sample_buoy_id_meteo].get(meteo_key)

    df_marine_name = "Marine DataFrame"
    df_meteo_name = "Météo DataFrame"

    if df_wanted is None or df_wanted.lower() == "marine":
        if sample_marine_df is not None and not sample_marine_df.empty:
            print(f"🌊 {df_marine_name} pour {sample_buoy_id_marine}:")
            print(sample_marine_df.info())
        else:
            print(f"⚠️ {df_marine_name} pour {sample_buoy_id_marine} : Aucune donnée")

    if df_wanted is None or df_wanted.lower() == "meteo":
        if sample_meteo_df is not None and not sample_meteo_df.empty:
            print(f"☁️ {df_meteo_name} pour {sample_buoy_id_meteo}:")
            print(sample_meteo_df.info())
        else:
            print(f"⚠️ {df_meteo_name} pour {sample_buoy_id_meteo} : Aucune donnée")

def display_buoys_missing_df_counts(buoy_datas, prefix=None, df_wanted=None):
    total = len(buoy_datas)
    marine_empty = 0
    meteo_empty = 0

    marine_key = f"{prefix} Marine" if prefix else "Marine"
    meteo_key = f"{prefix} Meteo" if prefix else "Meteo"

    for buoy_id, data in buoy_datas.items():
        df_marine = data.get(marine_key)
        df_meteo = data.get(meteo_key)

        if (df_wanted is None or df_wanted.lower() == "marine") and (df_marine is None or df_marine.empty):
            marine_empty += 1

        if (df_wanted is None or df_wanted.lower() == "meteo") and (df_meteo is None or df_meteo.empty):
            meteo_empty += 1

    if df_wanted is None or df_wanted.lower() == "marine":
        print(f"\n🌊 Nombre de bouées sans données '{marine_key}' : {marine_empty}/{total}")

    if df_wanted is None or df_wanted.lower() == "meteo":
        print(f"\n☁️ Nombre de bouées sans données '{meteo_key}' : {meteo_empty}/{total}")


Filter Buoys by Remarks

In [6]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        pass
# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Avant Filtre: 146
Après Filtre: 43


Build Buoys_datas Dict

In [7]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : south bermuda
🆔 Station ID : 41049
✅ Coordonnées extraites : Latitude = 27.50N, Longitude = 62.27W
🌊 Water Depth : 5480 m
🌡️ Sea Temp Depth : 1.5
🌬️ Barometer Elevation : 2.7
💨 Anemometer 

{'station_zone': 'grays reef',
 'lat_buoy': '31.40N',
 'lon_buoy': '80.87W',
 'Water_depth': '16 m',
 'sea_temp_depth': '2',
 'Barometer_elevation': '2.4',
 'Anemometer_height': '3.8',
 'Air_temp_height': '3.4',
 'url': 'https://www.ndbc.noaa.gov/station_page.php?station=41008'}

Collecte de données marines et météos

In [8]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Marine"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
    
        buoy_datas[buoy_id]["Meteo"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

# Retirer les bouées avec des DataFrames vides ou None
buoy_datas = {buoy_id: data for buoy_id, data in buoy_datas.items() 
              if "Marine" in data and data["Marine"] is not None and not data["Marine"].empty
              and "Meteo" in data and data["Meteo"] is not None and not data["Meteo"].empty}

# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")

# Afficher la longueur du dictionnaire (nombre de bouées avec des données valides)
print(f"\n📊 Nombre de bouées avec des données valides : {len(buoy_datas)}")


🚀 Démarrage du processus de collecte des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes renommées : {'date': 'Datetime'}

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !

📊 station_zone : ne st martin
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes

In [9]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


Data Enrichment with MetaDatas

In [10]:
list_not_include = ['lon_buoy', "lat_buoy", "url"]
for buoy_id, value in buoy_datas.items():
    print(f"\n🔍 Traitement de la Station ID: {buoy_id}")

    marine_df = buoy_datas[buoy_id]["Marine"]
    meteo_df = buoy_datas[buoy_id]["Meteo"]

    try:
        # Récupérer les métadonnées de la station
        buoy_metadata = get_station_metadata(buoy_id)
        parsed_data = parse_buoy_json(buoy_metadata)

        # Mise à jour du dictionnaire avec les métadonnées
        data = buoy_datas[buoy_id]
        data.update(parsed_data)
        
        # Ajouter les métadonnées comme nouvelles colonnes dans marine_df
        if marine_df is not None:
            marine_df["Station ID"] = str(buoy_id)
            for key, value in parsed_data.items():
                # Vérifier si la clé n'est pas dans la liste des exclusions
                if key not in list_not_include:
                    marine_df[key] = value
                    print(f"✅ Colonne '{key}' ajoutée au DataFrame de la station {buoy_id}")

    except Exception as e:
        print(f"❌ Erreur pour la station {buoy_id}: {e}")

# Vérification de l'ajout des colonnes en prenant un id au hasard
station_id = random.choice(list(buoy_datas.keys()))
marine_df = buoy_datas[station_id]["Marine"]

if marine_df is not None:
    print("\nColonnes ajoutées au DataFrame de la station", station_id)
    print(marine_df.columns)


🔍 Traitement de la Station ID: 41008

🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

✅ Colonne 'station_zone' ajoutée au DataFrame de la station 41008
✅ Colonne 'Water_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'sea_temp_depth' ajoutée au DataFrame de la station 41008
✅ Colonne 'Barometer_elevation' ajoutée au DataFrame de la station 41008
✅ Colonne 'Anemometer_height' ajoutée au DataFrame de la station 41008
✅ Colonne 'Air_temp_height' ajoutée au DataFrame de la station 41008

🔍 Traitement de la Station ID: 41044

🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, 

In [11]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


In [12]:
important_columns_oceanography = [
    'wind_direction',             
    'wind_speed',                 
    'wave_height',                   
    'pressure',                   
    'air_temperature',            
    'water_temperature',          
    'Datetime',
    'Lat',
    'Lon'                 
]

important_columns_meteorology = [
    'temperature_2m',             
    'relative_humidity_2m',       
    'dew_point_2m',               
    'precipitation',              
    'pressure_msl',               
    'cloud_cover',                
    'wind_speed_10m',             
    'Datetime'
]

Handle Null Values

In [13]:
stations_depart = len(buoy_datas)
ignored_buoys = {}  # Dictionary to track ignored buoys and their reasons

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Nettoyage des données pour la station {station_id}")

    marine_df = data.get("Marine")
    meteo_df = data.get("Meteo")

    if marine_df is None or meteo_df is None:
        ignored_buoys[station_id] = "Marine DataFrame ou Meteo DataFrame manquant(e)"
        print(f"⚠️ Station {station_id} ignorée: Marine DataFrame ou Meteo DataFrame manquant(e)")
        continue

    try:
        # Nettoyage des DataFrames
        cleaned_marine_df = handle_null_values(marine_df)
        cleaned_meteo_df = handle_null_values(meteo_df)
        # Vérification des colonnes importantes après nettoyage
        marine_columns_ok = all(col in cleaned_marine_df.columns for col in important_columns_oceanography)
        meteo_columns_ok = all(col in cleaned_meteo_df.columns for col in important_columns_meteorology)

        # Track which columns are missing
        missing_marine_columns = [col for col in important_columns_oceanography if col not in cleaned_marine_df.columns]
        missing_meteo_columns = [col for col in important_columns_meteorology if col not in cleaned_meteo_df.columns]

        if missing_marine_columns or missing_meteo_columns:
            ignored_buoys[station_id] = f"Colonnes manquantes: Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}"
            print(f"⚠️ Station {station_id} ignorée: Colonnes manquantes - Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}")
            continue

        # Ajouter le DataFrame nettoyé au dictionnaire des résultats
        buoy_datas[station_id]['Cleaned Marine'] = cleaned_marine_df
        buoy_datas[station_id]['Cleaned Meteo'] = cleaned_meteo_df
        print(f"✅ Nettoyage réussi pour la station {station_id} ({cleaned_marine_df.shape[0]} lignes)")

    except Exception as e:
        ignored_buoys[station_id] = f"Erreur lors du nettoyage: {e}"
        print(f"❌ Erreur lors du nettoyage pour {station_id}: {e}")

# 🔥 Suppression des stations ignorées du dictionnaire principal
for station_id in ignored_buoys:
    buoy_datas.pop(station_id, None)

len_cleaned_data = len([data for data in buoy_datas.values() if 'Cleaned Marine' in data and 'Cleaned Meteo' in data])

# Résumé final du nettoyage
print("\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"📌 Stations au départ : {stations_depart}")
print(f"✅ Stations nettoyées : {len_cleaned_data}")
print(f"🏁 Stations restantes après filtrage :")

for station_id, reason in ignored_buoys.items():
    print(f"🛑 Station {station_id} ignorée: {reason}")

print(f"\n🧹 Clés restantes dans buoy_datas après purge : {len(buoy_datas)} (attendu : {len_cleaned_data})")


🔄 Nettoyage des données pour la station 41008

Tag: orange - Nombre de lignes: 6537
Colonne 'wind_direction' Imputée par la médiane (26.3% de valeurs manquantes)
Colonne 'wind_speed' Imputée par la médiane (26.04% de valeurs manquantes)
Colonne 'wind_gust' Imputée par la médiane (26.04% de valeurs manquantes)
Colonne 'wave_height' Imputée par la médiane (49.06% de valeurs manquantes)
Colonne 'dominant_wave_period' Supprimée (plus de 55% de valeurs manquantes)
Colonne 'average_wave_period' Imputée par la médiane (49.07% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (49.11% de valeurs manquantes)
Colonne 'pressure' Imputée par la médiane (0.17% de valeurs manquantes)
Colonne 'air_temperature' Imputée par la médiane (26.04% de valeurs manquantes)
Colonne 'water_temperature' Imputée par la médiane (16.63% de valeurs manquantes)
Colonne 'dewpoint' Imputée par la médiane (26.39% de valeurs manquantes)
Colonne 'visibility' Supprimée (plus de 55% de valeurs m

In [14]:
display_buoys_missing_df_counts(buoy_datas, prefix="Cleaned")


🌊 Nombre de bouées sans données 'Cleaned Marine' : 0/23

☁️ Nombre de bouées sans données 'Cleaned Meteo' : 0/23


In [15]:
# Fusion des DataFrames nettoyés
print("\n🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION")

merged_success_count = 0  # Compteur de fusions réussies
total_merged_rows = 0     # Total de lignes fusionnées

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Fusion des données pour la station {station_id}")

    cleaned_marine_df = data.get("Cleaned Marine")
    cleaned_meteo_df = data.get("Cleaned Meteo")

    if cleaned_marine_df is None or cleaned_meteo_df is None:
        continue

    try:
        merged_df = pd.merge(cleaned_marine_df, cleaned_meteo_df, on="Datetime", how="inner")

        if merged_df.empty:
            print(f"⚠️ Station {station_id} fusionnée, mais résultat vide après inner merge sur 'Datetime'")
        else:
            buoy_datas[station_id]["Merged"] = merged_df
            merged_success_count += 1
            total_merged_rows += len(merged_df)
            print(f"✅ Fusion réussie pour la station {station_id} ({merged_df.shape[0]} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la fusion pour {station_id}: {e}")

# Résumé des fusions
print(f"\n📦 Fusions réussies : {merged_success_count}/{len_cleaned_data} stations")
print(f"📊 Total de lignes fusionnées : {total_merged_rows}")


🔗 FUSION DES DONNÉES MARINE + METEO PAR STATION

🔄 Fusion des données pour la station 41008
✅ Fusion réussie pour la station 41008 (1096 lignes)

🔄 Fusion des données pour la station 41044
✅ Fusion réussie pour la station 41044 (1096 lignes)

🔄 Fusion des données pour la station 41049
✅ Fusion réussie pour la station 41049 (1096 lignes)

🔄 Fusion des données pour la station 42001
✅ Fusion réussie pour la station 42001 (635 lignes)

🔄 Fusion des données pour la station 42002
✅ Fusion réussie pour la station 42002 (659 lignes)

🔄 Fusion des données pour la station 42056
✅ Fusion réussie pour la station 42056 (1096 lignes)

🔄 Fusion des données pour la station 42058
✅ Fusion réussie pour la station 42058 (1093 lignes)

🔄 Fusion des données pour la station 44007
✅ Fusion réussie pour la station 44007 (1095 lignes)

🔄 Fusion des données pour la station 44020
✅ Fusion réussie pour la station 44020 (1095 lignes)

🔄 Fusion des données pour la station 44065
✅ Fusion réussie pour la station 440

In [16]:
# Concaténation des DataFrames fusionnés
print("\n🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME")

final_merged_df_list = []
concat_success_count = 0
concat_total_rows = 0

for station_id, data in buoy_datas.items():
    merged_df = data.get("Merged")

    if merged_df is None:
        print(f"⚠️ Station {station_id} ignorée pour concaténation: Données fusionnées manquantes")
        continue

    try:
        final_merged_df_list.append(merged_df)
        concat_success_count += 1
        concat_total_rows += len(merged_df)
        print(f"✅ Concaténation réussie pour la station {station_id} ({len(merged_df)} lignes)")

    except Exception as e:
        print(f"❌ Erreur lors de la concaténation pour {station_id}: {e}")

# Création du DataFrame final unique
try:
    df_final = pd.concat(final_merged_df_list, ignore_index=True)
    print(f"\n🧾 DataFrame final créé avec succès ({df_final.shape[0]} lignes, {df_final.shape[1]} colonnes)")
except Exception as e:
    print(f"\n❌ Erreur lors de la création du DataFrame final: {e}")
    df_final = None

# Résumé
print(f"\n📦 Concaténations réussies : {concat_success_count}/{merged_success_count}")
print(f"📊 Total de lignes dans le DataFrame final : {concat_total_rows}")


🧬 CONCATÉNATION DES DONNÉES FUSIONNÉES EN UN SEUL DATAFRAME
✅ Concaténation réussie pour la station 41008 (1096 lignes)
✅ Concaténation réussie pour la station 41044 (1096 lignes)
✅ Concaténation réussie pour la station 41049 (1096 lignes)
✅ Concaténation réussie pour la station 42001 (635 lignes)
✅ Concaténation réussie pour la station 42002 (659 lignes)
✅ Concaténation réussie pour la station 42056 (1096 lignes)
✅ Concaténation réussie pour la station 42058 (1093 lignes)
✅ Concaténation réussie pour la station 44007 (1095 lignes)
✅ Concaténation réussie pour la station 44020 (1095 lignes)
✅ Concaténation réussie pour la station 44065 (1097 lignes)
✅ Concaténation réussie pour la station 46006 (1093 lignes)
✅ Concaténation réussie pour la station 46014 (1096 lignes)
✅ Concaténation réussie pour la station 46025 (1095 lignes)
✅ Concaténation réussie pour la station 46027 (1094 lignes)
✅ Concaténation réussie pour la station 46072 (1095 lignes)
✅ Concaténation réussie pour la station 4

In [17]:
display(df_final.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24296 entries, 0 to 24295
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   wind_direction           24296 non-null  float64       
 1   wind_speed               24296 non-null  float64       
 2   wind_gust                24296 non-null  float64       
 3   wave_height              24296 non-null  float64       
 4   average_wave_period      24296 non-null  float64       
 5   dominant_wave_direction  23201 non-null  float64       
 6   pressure                 24296 non-null  float64       
 7   air_temperature          24296 non-null  float64       
 8   water_temperature        24296 non-null  float64       
 9   dewpoint                 24296 non-null  float64       
 10  Datetime                 24296 non-null  datetime64[ns]
 11  Lat                      24296 non-null  object        
 12  Lon                      24296 n

None

Hour Filtering

In [18]:
try:
    df_final['Datetime'] = pd.to_datetime(df_final['Datetime'])

    df_final = df_final[df_final['Datetime'].dt.minute == 0]
    # Affichage pour vérifier le résultat
    print(f"🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: {df_final.shape[0]} lignes")

except Exception as e:
    print(f"�� Erreur lors du filtrage des lignes à l'heure pile: {e}")

🚀 DataFrame filtrée pour ne garder que les lignes à l'heure pile: 24296 lignes


In [19]:
final_datetime = pd.DataFrame(df_final[['Datetime']], dtype='datetime64[ns]')
final_datetime

,Datetime
0,2025-04-08 16:00:00
1,2025-04-08 15:00:00
2,2025-04-08 14:00:00
3,2025-04-08 13:00:00
4,2025-04-08 12:00:00
...,...
24291,2025-02-22 04:00:00
24292,2025-02-22 03:00:00
24293,2025-02-22 02:00:00
24294,2025-02-22 01:00:00


In [20]:
df_final = handle_null_values(df_final)


Tag: yellow - Nombre de lignes: 24296
Colonne 'wind_direction' non modifiée (0% de valeurs manquantes)
Colonne 'wind_speed' non modifiée (0% de valeurs manquantes)
Colonne 'wind_gust' non modifiée (0% de valeurs manquantes)
Colonne 'wave_height' non modifiée (0% de valeurs manquantes)
Colonne 'average_wave_period' non modifiée (0% de valeurs manquantes)
Colonne 'dominant_wave_direction' Imputée par la médiane (4.51% de valeurs manquantes)
Colonne 'pressure' non modifiée (0% de valeurs manquantes)
Colonne 'air_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'water_temperature' non modifiée (0% de valeurs manquantes)
Colonne 'dewpoint' non modifiée (0% de valeurs manquantes)
Colonne 'Datetime' non modifiée (0% de valeurs manquantes)
Colonne 'Lat' non modifiée (0% de valeurs manquantes)
Colonne 'Lon' non modifiée (0% de valeurs manquantes)
Colonne 'Water_depth' non modifiée (0% de valeurs manquantes)
Colonne 'Station ID' non modifiée (0% de valeurs manquantes)
Colonne 'stati

In [21]:
display_row_values(df_final)

wind_direction  |  wind_speed  |  wind_gust  |  wave_height  |  average_wave_period  |  dominant_wave_direction  |  pressure  |  air_temperature  |  water_temperature  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water_depth  |  Station ID  |  station_zone                              |  sea_temp_depth  |  Barometer_elevation  |  Anemometer_height  |  Air_temp_height  |  temperature_2m  |  relative_humidity_2m  |  dew_point_2m  |  precipitation  |  rain  |  showers  |  pressure_msl  |  surface_pressure  |  cloud_cover  |  cloud_cover_low  |  cloud_cover_mid  |  cloud_cover_high  |  visibility  |  wind_speed_10m  |  soil_temperature_0cm  |  soil_moisture_0_to_1cm  |  is_day
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
# get station id distinct values
station_ids = df_final['Station ID'].unique()
print(station_ids)

['41008' '41044' '41049' '42001' '42002' '42056' '42058' '44007' '44020'
 '44065' '46006' '46014' '46025' '46027' '46072' '46078' '46084' '46086'
 '46087' '46088' '51000' '51001' '51002']


In [23]:
df_final.columns = [col.strip() for col in df_final.columns]

In [24]:
# Dictionnaire de renommage des colonnes
col_to_rename = {'temperature_2m': 'T°(C°)', 'relative_humidity_2m': 'Relative Humidity (%)',
                 'dew_point_2m': 'Dew Point (°C)', 'precipitation': 'Precipitations (mm)',  
                 'pressure_msl':' Sea Level Pressure (hPa)', 'cloud_cover_low':'Low Clouds (%)', 
                 'cloud_cover_mid' : 'Middle Clouds (%)', 'cloud_cover_high' : 'High Clouds (%)', 
                 'visibility' : ' Visibility (km)', 'wind_direction': 'Wind Direction (°)',
                 'wind_speed': 'Wind Speed (km/h)', 'wind_gust': 'Wind Gusts (km/h)',
                 'wind_speed_10m':'Wind Speed (10m)', 'surface_pressure': 'Surface Pressure','wave_height': 'Wave Height (m)', 'average_wave_period': 'Average Wave Period (s)',
                 'dominant_wave_direction': 'Dominant Wave Direction (°)', 'pressure': 'Pressure (hPa)',
                 'air_temperature': 'Air T°', 'water_temperature': 'Water T°', 'Water_depth': 'Water Depth (m)',
                 "Air_temp_height": "Air T° Height", "Anemometer_height": "Anemometer Height (m)",
                 "station_zone": "Station Zone",
                 "Barometer_elevation": "Barometer Elevation", "sea_temp_depth" : "Sea Temperature Depth (m)"}
# Liste des colonnes à supprimer
cols_to_delete = ['soil_temperature_0cm', 'lat_buoy','lon_buoy' 'rain', 'showers', 'is_day', 'soil_moisture_0_to_1cm', 'cloud_cover']

# Renommer les colonnes d'abord
df_final = rename_columns(df_final, col_to_rename)
# Ensuite, supprimer les colonnes non désirées
df_final = drop_columns_if_exist(df_final, cols_to_delete)

# Afficher les résultats
print("\nColonnes après renommage et suppression :")
print(df_final.columns)

🔄 Colonne 'temperature_2m' renommée en 'T°(C°)'
🔄 Colonne 'relative_humidity_2m' renommée en 'Relative Humidity (%)'
🔄 Colonne 'dew_point_2m' renommée en 'Dew Point (°C)'
🔄 Colonne 'precipitation' renommée en 'Precipitations (mm)'
🔄 Colonne 'pressure_msl' renommée en ' Sea Level Pressure (hPa)'
🔄 Colonne 'cloud_cover_low' renommée en 'Low Clouds (%)'
🔄 Colonne 'cloud_cover_mid' renommée en 'Middle Clouds (%)'
🔄 Colonne 'cloud_cover_high' renommée en 'High Clouds (%)'
🔄 Colonne 'visibility' renommée en ' Visibility (km)'
🔄 Colonne 'wind_direction' renommée en 'Wind Direction (°)'
🔄 Colonne 'wind_speed' renommée en 'Wind Speed (km/h)'
🔄 Colonne 'wind_gust' renommée en 'Wind Gusts (km/h)'
🔄 Colonne 'wind_speed_10m' renommée en 'Wind Speed (10m)'
🔄 Colonne 'surface_pressure' renommée en 'Surface Pressure'
🔄 Colonne 'wave_height' renommée en 'Wave Height (m)'
🔄 Colonne 'average_wave_period' renommée en 'Average Wave Period (s)'
🔄 Colonne 'dominant_wave_direction' renommée en 'Dominant Wave 

In [25]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Air T°  |  Water T°  |  dewpoint  |  Datetime             |  Lat     |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  rain  |   Sea Level Pressure (hPa)  |  Surface Pressure  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |   Visibility (km)  |  Wind Speed (10m)
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [26]:
try:
    df_final[['Lat', 'Lon']] = df_final.apply(
        lambda row: pd.Series(convert_coordinates(row['Lat'], row['Lon'])),
        axis=1
    )
except Exception as e:
    print(f"❌ Erreur lors de la conversion des coordonnées : {e}")


In [27]:
#  T°(C°) = (Air T° + T°(C°))/2 et virer Air T°
try:
    df_final['T°(C°)'] = (df_final['Air T°'] + df_final['T°(C°)']) / 2
    df_final.drop(columns=['Air T°'], inplace=True)
except:
    pass

# Sea Level Pressure (hPa) = (Sea Level Pressure (hPa) +Surface Pressure)/2 et virer Surface Pressure

try:
    df_final['Sea Level Pressure (hPa)'] = (df_final['Sea Level Pressure (hPa)'] + df_final['Surface Pressure']) / 2
    df_final.drop(columns=['Surface Pressure'], inplace=True)
except:
    pass

# virer le m dans Water Depth avec regex lambda et passer la colonne en float
df_final['Water Depth (m)'] = df_final['Water Depth (m)'].apply(lambda x: re.sub(r'\D', '', str(x)).strip())
df_final['Water Depth (m)'].astype(float)

0          16.0
1          16.0
2          16.0
3          16.0
4          16.0
          ...  
24291    4979.0
24292    4979.0
24293    4979.0
24294    4979.0
24295    4979.0
Name: Water Depth (m), Length: 24296, dtype: float64

In [28]:
display_row_values(df_final)

Wind Direction (°)  |  Wind Speed (km/h)  |  Wind Gusts (km/h)  |  Wave Height (m)  |  Average Wave Period (s)  |  Dominant Wave Direction (°)  |  Pressure (hPa)  |  Water T°  |  dewpoint  |  Datetime             |  Lat    |  Lon      |  Water Depth (m)  |  Station ID  |  Station Zone                              |  Sea Temperature Depth (m)  |  Barometer Elevation  |  Anemometer Height (m)  |  Air T° Height  |  T°(C°)                  |  Relative Humidity (%)  |  Dew Point (°C)  |  Precipitations (mm)  |  rain  |   Sea Level Pressure (hPa)  |  Surface Pressure  |  Low Clouds (%)  |  Middle Clouds (%)  |  High Clouds (%)  |   Visibility (km)  |  Wind Speed (10m)
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Check truth about Wind Speed Using another API call

In [29]:
df_42058 = df_final[df_final['Station ID'] == "42058"]
df_42058.columns

Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPa)', 'Water T°', 'dewpoint',
       'Datetime', 'Lat', 'Lon', 'Water Depth (m)', 'Station ID',
       'Station Zone', 'Sea Temperature Depth (m)', 'Barometer Elevation',
       'Anemometer Height (m)', 'Air T° Height', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitations (mm)',
       'rain', ' Sea Level Pressure (hPa)', 'Surface Pressure',
       'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
       ' Visibility (km)', 'Wind Speed (10m)'],
      dtype='object')

In [30]:
# ---- Chargement de la clé API ----
vc_api_key_path = r"c:\Credentials\visual_crossing_weather_api.json"
with open(vc_api_key_path, 'r') as file:
    content = json.load(file)
    vc_api_key = content["api_key"]

# ---- Extraire les coordonnées depuis la première ligne du DataFrame ----
lat_42058, lon_42058 = None, None
if not df_42058.empty:
    first_row = df_42058.iloc[0]
    lat_42058, lon_42058 = first_row["Lat"], first_row["Lon"]

# ---- Définir les dates pour la requête ----
today = datetime.now().strftime("%Y-%m-%d")
last_month = (datetime.now() - timedelta(days=31)).strftime("%Y-%m-%d")

# ---- Créer le dossier de cache si nécessaire ----
cache_dir = "api_call_files"
os.makedirs(cache_dir, exist_ok=True)

# ---- Définir le fichier cache selon la position ----
cache_file = os.path.join(cache_dir, f"vc_meteo_{lat}_{lon}.csv")

# ---- Vérifier si un cache récent existe (moins de 24h) ----
use_cache = False
if os.path.exists(cache_file):
    last_modified = datetime.fromtimestamp(os.path.getmtime(cache_file))
    if datetime.now() - last_modified < timedelta(hours=24):
        print(f"📦 Cache détecté ({cache_file}), modifié le {last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
        vc_meteo_df = pd.read_csv(cache_file)
        print("✅ Données météo rechargées depuis le cache.")
        use_cache = True
    else:
        print(f"⚠️ Cache trouvé mais périmé (plus de 24h) → nouvelle requête API.")

# ---- Appel API si pas de cache valide ----
if not use_cache and lat is not None and lon is not None:
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{lat},{lon}/{last_month}/{today}?unitGroup=metric&key={vc_api_key}&contentType=json"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            vc_meteo_data = response.json()
            print("🌍 Données météo récupérées depuis l'API Visual Crossing.")

            # ---- Extraire les données journalières et sauvegarder ----
            if "days" in vc_meteo_data:
                vc_meteo_df = pd.json_normalize(vc_meteo_data["days"])
                vc_meteo_df.to_csv(cache_file, index=False)
                print(f"💾 Données sauvegardées dans le cache : {cache_file}")
            else:
                print("⚠️ Le champ 'days' est absent de la réponse API.")
        else:
            print(f"❌ Échec de l’appel API — code de statut : {response.status_code}")
    except Exception as e:
        print(f"❌ Exception levée lors de la requête API : {e}")

NameError: name 'lat' is not defined

In [ ]:
# 📁 Charger les données du CSV Visual Crossing
vc_csv_path = f"api_call_files/vc_meteo_{lat_42058}_{lon_42058}.csv"
df_vc_meteo = pd.read_csv(vc_csv_path)
df_vc_meteo.head()

In [ ]:
# 🧼 Nettoyage et transformation
import ast  # Permet de convertir les strings représentant des listes en objets Python
all_hours = []

# Parcours de chaque ligne du fichier météo
for i, row in df_vc_meteo.iterrows():
    try:
        # Convertir la colonne "hours" en liste de dictionnaires
        hours_list = ast.literal_eval(row['hours'])

        # Pour chaque heure dans la journée, on ajoute une entrée au tableau final
        for hour_data in hours_list:
            hour_data['Date'] = row['datetime']  # On ajoute aussi la date du jour correspondant
            all_hours.append(hour_data)

    except Exception as e:
        print(f"Erreur parsing ligne {i}: {e}")  # En cas d'erreur de parsing (souvent problème de format)

# ✅ On transforme la liste aplanie en DataFrame
df_vc_flat = pd.DataFrame(all_hours)

# 🕒 Conversion de l'heure/temps
df_vc_flat["Datetime"] = pd.to_datetime(df_vc_flat["datetimeEpoch"], unit="s").dt.strftime("%Y-%m-%d-%H")

# 🗓️ Filtrer sur les 30 derniers jours
today = datetime.now()
thirty_days_ago = today - timedelta(days=30)

today_str = today.strftime("%Y-%m-%d")
thirty_days_ago_str = thirty_days_ago.strftime("%Y-%m-%d")

# On filtre le DataFrame pour ne garder que les lignes entre ces deux dates
df_vc_last_month = df_vc_flat[
    (df_vc_flat['Date'] >= thirty_days_ago_str) & 
    (df_vc_flat['Date'] <= today_str)]

In [ ]:
display_row_values(df_vc_last_month)

In [ ]:
# Correct syntaxe pour filtrer les colonnes spécifiques
df_vc_last_month = df_vc_last_month[["Datetime", "temp", "humidity", "precip", "dew", "windgust", "windspeed", "winddir", "pressure", "visibility"]]

for col in df_vc_last_month.columns:
    if not col == "Datetime":
        rename_columns(df_vc_last_month, {col: f"VC_{col}"})

In [ ]:
#merge the df_42058 with df_vc_last_month to compare

df_merged = pd.merge(df_42058, df_vc_last_month, on="Datetime", how="inner")
df_merged.head()

Map Visualization

In [ ]:
# list_coords=[]

# for id in buoy_list:
#     metadata = get_station_metadata(id)
    
#     station_id, station_zone, lat_buoy, lon_buoy = parse_buoy_json(metadata)
#     lat_buoy, lon_buoy = convert_coordinates(lat_buoy,lon_buoy)
#     coords = [station_id, station_zone, lat_buoy, lon_buoy]
#     list_coords.append(coords)

# def barycentre(coords):
#     x_coords = [item[2] for item in coords]
#     y_coords = [item[3] for item in coords]

#     x_barycentre = sum(x_coords) / len(coords)
#     y_barycentre = sum(y_coords) / len(coords)

#     return [x_barycentre, y_barycentre]

# # Exemple d'utilisation
# coords = [[1, 2], [3, 4], [5, 6], [7, 8]]
# centre = barycentre(list_coords)
# print(list_coords)
# print("Coordonnées du barycentre :", centre)

In [ ]:
# map = folium.Map(location=(centre[0],centre[1]), 
#                  tiles="Esri.WorldImagery", 
#                  zoom_start=2.5, attr="Données fournies par Esri")
# for loc in list_coords:
#     lat = loc[2]
#     lon = loc[3]
#     station_id = loc[0]
#     station_zone = loc[1]
#     folium.Marker(
#     location=[lat, lon],
#     popup=f"Station ID: {station_id}\n\nZone: {station_zone}\nLat: {lat}\n\nLon: {lon}",
#     icon=folium.Icon(icon="cloud"),
# ).add_to(map)

# map

Columns Check Tests

In [ ]:
choice = random.choice(buoy_list)
choice

In [ ]:
df_marine_test = NDBC.realtime_observations(choice)
print(f'{df_marine_test.shape[0]} rows \n\n{df_marine_test.isna().sum()}')
df_marine_test.head(1)

Check test API Open-Meteo

In [ ]:
df_meteo_test = meteo_api_request(coordinates=[12, 23])
print(f'{df_meteo_test.shape[0]} rows \n{df_meteo_test.isna().sum()}')
df_meteo_test.head(1)

In [ ]:
metadata_extracted = get_station_metadata(choice)
metadata_extracted

In [ ]:
# def drop_schema_or_tables(conn, schema_name, drop_schema=False):
#     try:
#         if drop_schema:
#             # Supprimer le schéma si drop_schema est True
#             print(f"Dropping schema {schema_name}...")
#             conn.execute(text(f"DROP SCHEMA IF EXISTS \"{schema_name}\" CASCADE;"))
#             conn.commit()
#             print(f"Schema {schema_name} dropped successfully.")
#         else:
#             # Supprimer toutes les tables dans le schéma
#             print(f"Dropping all tables in schema {schema_name}...")
#             conn.execute(text(f"""
#                 DO $$ 
#                 DECLARE
#                     r RECORD;
#                 BEGIN
#                     FOR r IN 
#                         SELECT table_name 
#                         FROM information_schema.tables 
#                         WHERE table_schema = :schema_name
#                     LOOP
#                         EXECUTE 'DROP TABLE IF EXISTS "' || :schema_name || '".' || quote_ident(r.table_name) || ' CASCADE';
#                     END LOOP;
#                 END $$;
#             """), {'schema_name': schema_name})
#             conn.commit()
#             print(f"All tables in schema {schema_name} dropped successfully.")
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         conn.rollback()
#         print("Transaction rolled back.")

# drop_schema_or_tables(conn=conn, schema_name=schema, drop_schema=False)

In [ ]:
# def explore_dict(d, indent=0):
#     """ Fonction récursive pour afficher toute la structure du dictionnaire """
#     for key, value in d.items():
#         print(" " * indent + f"- {key}: {type(value)}")
#         if isinstance(value, dict):
#             explore_dict(value, indent + 4)  # Explorer récursivement avec une indentation
#         elif isinstance(value, list):
#             if len(value) > 0:
#                 print(" " * (indent + 4) + f"Liste ({len(value)} éléments), type du premier élément: {type(value[0])}")
#                 if isinstance(value[0], dict):
#                     explore_dict(value[0], indent + 8)  # Explorer si c'est une liste de dicts
#         else:
#             print(" " * (indent + 4) + f"Valeur: {value}")

# # Récupérer la première clé du dictionnaire
# first_key = next(iter(buoy_datas))
# print(f"Exploration de la première clé: {first_key}\n")

# # Exécuter la fonction sur le premier élément uniquement
# explore_dict({first_key: buoy_datas[first_key]})

In [ ]:
# openmongo_creds = r'C:\Users\f.gionnane\Documents\Data Engineering\Credentials\mongo_creds.json'

# with open(openmongo_creds, 'r') as file:
#     content = json.load(file)
#     mongo_user = content["user"]
#     mongo_password = content["password"]
#     mongo_string = content["connection_string"]

# uri = mongo_string
# # Create a new client and connect to the server
# client = MongoClient(uri, server_api=ServerApi('1'))

# # Send a ping to confirm a successful connection
# try:
#     client.admin.command('ping')
#     print("Pinged your deployment. You successfully connected to MongoDB!")
# except Exception as e:
#     print(e)
#     print(f'{mongo_user}\n{mongo_password}\n{mongo_string}')

In [ ]:
# Charger le fichier GeoJSON depuis l'URL
url = "https://gist.githubusercontent.com/jrrickard/8755532505a40f3b8317/raw/ecd98849d3a5f4502b773b986254f19af3b8d8fb/oceans.json"
geojson_data = requests.get(url).json()

# Créer une carte avec un fond satellite ESRI
m = folium.Map(location=[0, 0], zoom_start=3, tiles="Esri.WorldImagery")

# Ajouter le GeoJSON (les océans) à la carte
folium.GeoJson(geojson_data, name="Oceans").add_to(m)

# Fonction pour générer des coordonnées aléatoires dans l'océan
def random_ocean_coords():
    lat = random.uniform(-60, 60)  # Latitude dans les eaux de l'océan
    lon = random.uniform(-180, 180)  # Longitude dans les eaux de l'océan
    return lat, lon

# Générer un DataFrame avec des coordonnées, températures, hauteur des vagues, vitesse du vent, etc.
data = {
    "Coordinates": [],
    "Temperature (°C)": [],
    "Wave Height (m)": [],
    "Wind Speed (km/h)": []
}

# Générer 10 marqueurs avec des valeurs aléatoires
for _ in range(10):
    lat, lon = random_ocean_coords()
    
    # Générer des valeurs aléatoires pour la température, la hauteur des vagues et la vitesse du vent
    temperature = random.uniform(15, 30)  # Température entre 15 et 30°C
    wave_height = random.uniform(0.5, 5)  # Hauteur des vagues entre 0.5m et 5m
    wind_speed = random.uniform(10, 50)  # Vitesse du vent entre 10 km/h et 50 km/h
    
    # Ajouter ces valeurs dans le DataFrame
    data["Coordinates"].append((lat, lon))
    data["Temperature (°C)"].append(temperature)
    data["Wave Height (m)"].append(wave_height)
    data["Wind Speed (km/h)"].append(wind_speed)

# Créer un DataFrame pandas avec ces données
df = pd.DataFrame(data)

# Ajouter les marqueurs et cercles à la carte avec des couleurs dépendant de la température
for index, row in df.iterrows():
    lat, lon = row["Coordinates"]
    temperature = row["Temperature (°C)"]
    
    # Déterminer la couleur du cercle en fonction de la température
    if temperature < 20:
        fill_color = "blue"
    elif temperature < 25:
        fill_color = "green"
    else:
        fill_color = "red"
    
    # Ajouter un marqueur
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)
    
    # Ajouter un cercle autour du marqueur
    folium.CircleMarker(
        location=[lat, lon],
        radius=15,  # Rayon du cercle
        color="black",
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.5
    ).add_to(m)

# Ajouter une couche de contrôle pour la carte
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save("ocean_markers_map.html")

# Affichage de la carte dans l'environnement interactif (si vous êtes dans un environnement Jupyter)


In [ ]:
m